**Goal:** represent the tracks downloaded from [Wikiloc](http://wikiloc.com/) or transformed into .gpx format from Garmin .fit files, using for example the [gpsbabel](http://www.gpsbabel.org/) tool.

In [36]:
import re
import os
import folium
import glob
import logging
import shutil
import numpy as np
from geopy.geocoders import Nominatim
from folium import plugins
import matplotlib.pyplot as plt
from matplotlib import colors

In [37]:
print folium.__version__

0.2.1


# User inputs

Define the directory where the files are stored and where the html will be written.

In [47]:
region = 'Mallorca/'
gpxdir = '/home/ctroupin/DataOceano/GPX/' + region
outputdir = '/home/ctroupin/public_html/LeafletMaps'
logdir = '/home/ctroupin/logs/'
mapname = region + '.html'
gpxbasename = '*.gpx'
gpxfilelist = sorted(glob.glob(os.path.join(gpxdir, gpxbasename)))

In [48]:
gpxfilelist[-10:]

['/home/ctroupin/DataOceano/GPX/Mallorca/soller-orient-santa-maria-del-cami.gpx',
 '/home/ctroupin/DataOceano/GPX/Mallorca/soller-pollenca.gpx',
 '/home/ctroupin/DataOceano/GPX/Mallorca/soller-puig-de-lofre.gpx',
 '/home/ctroupin/DataOceano/GPX/Mallorca/soller-tossals-verds-lloseta.gpx',
 '/home/ctroupin/DataOceano/GPX/Mallorca/son-vida-puig-des-revells.gpx',
 '/home/ctroupin/DataOceano/GPX/Mallorca/subida-al-puig-de-galatzo.gpx',
 '/home/ctroupin/DataOceano/GPX/Mallorca/tomir-42-2015.gpx',
 '/home/ctroupin/DataOceano/GPX/Mallorca/valldemossa-deia-por-el-teix.gpx',
 '/home/ctroupin/DataOceano/GPX/Mallorca/valldemossa-teix.gpx',
 '/home/ctroupin/DataOceano/GPX/Mallorca/vuelta-por-el-parc-natural-de-llevant.gpx']

In [49]:
if not(os.path.exists(logdir)):
    os.mkdir(logdir)
logger = logging.getLogger()
fhandler = logging.FileHandler(filename=os.path.join(logdir, 'GPXheadmap.log'), mode='a')
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fhandler.setFormatter(formatter)
logger.addHandler(fhandler)
logger.setLevel(logging.DEBUG)

Initialise the map

In [50]:
geolocator = Nominatim()
try:
    location = geolocator.geocode("Lloret de Vistalegre, Mallorca, Spain")
    #location = geolocator.geocode("Verviers, Belgique")
    #location = geolocator.geocode("Gran Canaria, Spain")
    #location = geolocator.geocode("La Palma, Spain")
    logger.info("Working on location: %s" %(location[0]))
except "GeocoderServiceError" or "GeocoderUnavailable":
    logger.error('Name or service not known')
    location.longitude = -15.
    location.longitude = 28.

In [51]:
location_longitude = -15.
location_latitude = 28.

Function to read GPX

In [52]:
def read_gpx(filename):
    coords = []
    with open(filename) as f:
        for lines in f:
            match = re.search('<trkpt lat="([-0-9\.]+)" lon="([-0-9\.]+)">', lines)
            if match:
                coords.append((float(match.group(1)), float(match.group(2))))
    return coords

In [53]:
def get_trackname(filename):
    '''
    Get the track name from a GPX file
    '''
    with open(filename) as f:
        match = None
        while match == None:
            line = f.readline()
            match = re.search('<name>(.+)</name>', line)
            if match:
                trackname = match.group(1)
    return trackname

In [54]:
def create_gradient(cb, ncolors):
    '''
    From a given matplotlib color palette, 
    provide a gradient dictionnary to be passed to 
    the heatmap function of folium
    INPUTS: cb = matplotlib colormap
            ncolors = number of colors
    OUTPUT: gradient = dictionnary containing the color
    '''
    gradient = {}
    for i, j in zip(np.linspace(0.35, 1.0, ncolors), np.linspace(0., cb.N-1, ncolors).astype('uint8')):
        gradient[i] = colors.rgb2hex(cb.colors[j])
    return gradient

Loop on the files

In [55]:
logger.info('Creating a new map')
map_run = folium.Map(location=[location.latitude, location.longitude], 
                     zoom_start=10)
iconurl = 'http://socib.es/users/ctroupin/LeafletMaps/glider.png'

logger.info('Create gradient dictionnary')
gradient = create_gradient(plt.cm.plasma, 10)
coordsall = []
logger.info('Loop on the GPX files')
for gpxfiles in gpxfilelist:
    logger.debug(gpxfiles)
    #trackname = get_trackname(gpxfiles)
    #logger.debug(trackname)
    logger.debug('Reading coordinates...')
    coords = read_gpx(gpxfiles)
    logger.debug('...finished reading coordinates')

    coordsall.append(coords)
    #folium.PolyLine(coords, color="black", weight=3).add_to(map_run)
    #icon = folium.features.CustomIcon(iconurl, icon_size=(50, 50))
    #folium.Marker(location=coords[0], popup=gpxfiles).add_to(map_run)

coords2plot = [item for coords in coordsall for item in coords] 
folium.plugins.HeatMap(coords2plot, radius=7, blur=5).add_to(map_run)
map_run

In [56]:
if __name__ == '__main__':
    main()

NameError: name 'main' is not defined

In [ ]:
if not(os.path.exists(outputdir)):
    os.makedirs(outputdir)
    logging.info('Creating directory %s' %(outputdir))
map_run.save(os.path.join(outputdir, mapname))

In [ ]:
outputdir